In [ ]:
%matplotlib notebook
%pylab
import pandas as pd
from time import perf_counter
from scipy.optimize import fsolve
from scipy.special import erf

In [ ]:
# import module that contains the functions and solver
from Module_ice_ocean import*

In [ ]:
def AllZeros(f,xmin,xmax,N):
    
    # Inputs :
    # f : function of one variable
    # [xmin - xmax] : range where f is continuous containing zeros
    # N : control of the minimum distance (xmax-xmin)/N between two zeros

    dx=(xmax-xmin)/N
    x2=xmin
    y2=f(x2)
    z=[]
    for i in range (1,N):
        x1=x2
        y1=y2
        x2=xmin+i*dx
        y2=f(x2)
        if (y1*y2<=0):                          
            
            z.append(fsolve(f,(x2*y1-x1*y2)/(y1-y2))[0])
    return array(z)

func = lambda x: tan(3*x) +2*x

In [ ]:
c = 1
def exactSol(x, t, icase):
    # dirichlet
    if(icase == 1):
        #ax = -1, bx = 1
        return (0.025/sqrt(0.000625+0.02*t))*exp(-((x+0.5-t)**2/(0.00125+0.04*t)))
    elif(icase == 2):
        #ax = 0, bx = 2
        p = 0.25 ; a = 0.1
        return (1/sqrt(1+t))*exp(-(x-(1+t)*p)**2/(4*a*(1+t)))
    elif(icase == 3): # Dirichlet
        # ax = 0, bx = 2*pi
        return exp(-t)*sin(x)
    elif(icase == 4):
        #ax = 0, bx = 1
        return sqrt(20/(20+t))*exp(-((x-2-0.8*t)**2)/(0.4*(t+20)))
    elif(icase == 5): # Neumann
        return exp(-pi**2*t)*cos(pi*x)
    elif(icase == 6):  # robin condition
        M = 5
        X = zeros(M)
        for n in range(1,M+1):

            z = AllZeros(func,(2*n-1)*pi/6,n*pi/3,100)
            X[n-1] = z[0]

        cn = 200*(3*X-sin(3*X))/(3*X*(3*X-sin(3*X)*cos(3*X)))

        un = 0

        for n in range(1, M+1):

            un += cn[n-1]*exp(-X[n-1]**2*t/25)*sin(X[n-1]*x)
        return un
    
def g(x, t, icase):
    
    # dirichlet
    if(icase == 1):
        return -2*(x+0.5-t)/(0.00125+0.04*t)*(0.025/sqrt(0.000625+0.02*t))*exp(-(x+0.5-t)**2/(0.00125+0.04*t))
    
    elif(icase == 2):
        p = 0.25 ; a = 0.1
        return -((x-(1+t)*p)/(2*a*(1+t)*sqrt(1+t)))*exp(-(x-(1+t)*p)**2/(4*a*(1+t)))
    elif(icase == 3):
        return exp(-t)*cos(x)
    elif(icase == 4):
        
        return -2*((x-2-0.8*t)/(0.4*(t+20)))*sqrt(20/(20+t))*exp(-((x-2-0.8*t)**2)/(0.4*(t+20)))
    elif(icase == 5):
        return -pi*exp(-pi**2*t)*sin(pi*x)
    elif(icase == 6):   # robin condition
        return -0.*exactSol(x, t, icase)
    
    
def domain(icase):
    if(icase == 1):
        ax = 0; bx = 1
    elif(icase == 2):
        ax = 0; bx = 2
    elif(icase == 3):
        ax = 0; bx = 2*pi
    elif(icase == 4):
        ax = 0; bx = 1
    elif(icase == 5):
        ax = 0; bx = 1
    elif(icase == 6):
        ax = 0; bx = 3
    print('Domain: [{}, {}]\n'.format(ax,bx))    
    return ax, bx
    
    
    
def coeff_advec_diffu(icase):
    
    if(icase == 1):
        c1 = 1 ; c = 0.01
        
    elif(icase == 2):
        c1 = 0.25 ; c = 0.1
    elif(icase == 3):
        c1 = 0 ; c = 1
    elif(icase == 4):
        c1 = 0.8 ; c = 0.1 
    elif(icase == 5):
        c1 = 0 ; c = 1
    elif(icase == 6):
        c1 = 0 ; c = 1/25
    # problem type
    if(c1 != 0 and c != 0):
        print('Problem type: Advection-diffusion\n')
    elif(c1 == 0 and c != 0):
        print('Problem type: Pure diffusion\n')
    if(c1 != 0 and c == 0):
        print('Problem type: Advection\n')
    return c1, c   

def boundary_conditions_coeff(icase):
    
    if(icase == 1):
        print('Boundary conditition type: Dirichlet\n')
        alpha = 0 ; beta = 1
    elif(icase == 2):
        print('Boundary conditition type: Neumann\n')
        alpha = 1 ; beta = 0
    elif(icase == 3):
        print('Boundary conditition type: Dirichlet\n')
        alpha = 0 ; beta = 1
    elif(icase == 4):
        print('Boundary conditition type: Dirichlet\n')
        alpha = 0 ; beta = 1
    elif(icase == 5):
        print('Boundary conditition type: Neumann\n')
        alpha = 1 ; beta = 0   
    elif(icase == 6):
        print('Boundary conditition type: Robin\n')
        alpha = 1 ; beta = 1/2   
    return alpha, beta

In [ ]:
def TempB(Sb, a,b,c,pb): 
    '''
    Sb: interface salinity
    pb: interface pressure
    a,b,c: constants
    TB = aSb + b + cpb 
    '''
    return a*Sb + b + c*pb

def Meltrate(Sw, Sb, gammaS): 
    '''
    Sw: ambient salinity
    Sb: interface salinity
    gammaS: salinity exchange velocity
    Melt rate: V = gammaS*(Sw - Sb)/Sb 
    '''
    return gammaS*(Sw - Sb)/Sb

def SaltB(K,L,M,Sw): 
    '''
    Interface salinity 
    '''
    D = L**2 - 4*K*M*Sw
    Sb1 = (-L + sqrt(D))/(2*K) 
    Sb2 = (-L - sqrt(D))/(2*K)
    return Sb1, Sb2


def coefK(a, gammaS, gammaT):
    return a*(1 - gammaS/gammaT)

def coefF(Tw, gammaS,gammaT, cw, Lf, cI, TS, b, c, pb, a, Sw):
    A = -Tw - (gammaS/(cw*gammaT))*(Lf - cI*TS) 
    B = (b + c*pb)*(1 - gammaS/gammaT)
    C = a*(gammaS/gammaT)*Sw
    return A + B + C

def coefM(gammaS, gammaT, cw, Lf, cI, TS, a, b, c, pb):
    A = (gammaS/(cw*gammaT))*(Lf - cI*TS) 
    B = a*(gammaS/gammaT)*(b + c*pb)
    return A + B

In [ ]:
#a = -5.73e-2     # Salinity coefficient of freezing equation(˚C psu^{-1})
a = -6e-2
b = 9.39e-2      # Constant coefficient of freezing equation(˚C)
c = -7.53e-8     # Pressure coefficient of freezing equation(˚C Pa^{-1})
cI = 2009.0      # Specific heat capacity ice(J kg^-1 K^-1)
cw = 3974.0      # Specific heat capacity water(J kg^-1 K^-1)
Lf = 3.348e+5    # Latent heat fusion(J kg^-1)
Tw = 5.4         # Temperature of water(˚C)
TS = -15        # Temperature of ice(˚C)
Sw = 35          # Salinity of water(psu)
Sc = 2500        # Schmidt number
Pr = 14          # Prandtl number
mu = 1.95e-6     # Kinematic viscosity of sea water(m^2 s^-1)
pb = 1.0e+7      # Pressure at ice interface(Pa)
kT = mu/Pr       # Thermal diffusivity(m^2 s^-1)
kS = mu/Sc       # Salinity diffusivity(m^2 s^-1)
rhoI = 920       # density of ice(kg m^-3)
rhoW = 1025      # density of sea water(kg m^-3)
gammaS = 5.05e-7 # Salinity exchange velocity(m s^-1)
gammaT = 1.0e-4  # Thermal exchange velocity(m s^-1)
rho = rhoI/rhoW  #

In [ ]:
# define exact, source functions
def initial_Temp(x, cst1):
    return cst1 + 9.5e-4*x

def initial_Salt(x, cst2):
    return cst2 + 4.0e-4*x
    
# Neumann boundary condition
def hT(bcst1, V):
    return rho*bcst1*V

def hB(bcst2,SB, V):
    return rho*bcst2*SB*V

K = coefK(a, gammaS, gammaT)
M = coefM(gammaS, gammaT, cw, Lf, cI, TS, a, b, c, pb)

In [ ]:
order = array([1,2,3,4])            # polynomial order
N_element = array([16,32,64,128])
Tw = 2.3
kstages = 3
Tfinal = 1
ti_method = 3
time_method = "BDF3"          # IRK = implicit RK method or BDF2
integration_type = 1          # % = 1 is inexact and = 2 is exact
method_type = 'cg'
iplot = False                 # plot the solution
icase = 5                     # case number: 1 is a Gaussian and 2 is a sinusoidal

text_case = 'ice-ocean'            # unit: ideal test case for convergence studies
                              # ice-ocean: for ice ocean simulation
    
if(text_case == 'unit'):

    # advection and diffusion coefficients
    c1, c = coeff_advec_diffu(icase)

    # Boundary conditition type                   

    # Robin: alpha = 1, beta != 0
    # Neumann: alpha = 1, beta = 0
    # Dirichlet: alpha = 0, beta = 1

    alpha, beta = boundary_conditions_coeff(icase)

    # Problem domain
    ax, bx = domain(icase)
    
elif(text_case == 'ice-ocean'):
    
    ax = 0
    bx = 0.05
    u1 = 0
    
    Tfinal = 100
    
    order = array([2])            # polynomial order
    N_element = array([20])

# Declaration of some variables
diss = 0
u = 1
nel0 = 8
len_el = len(N_element)
len_pol = len(order)
l2e_norm = zeros((len_pol, len_el))
max_norm = zeros((len_pol, len_el))

Np_array = zeros((len_pol, len_el))

Nv = N_element

# Simulation

for iN,N in enumerate(order):
    
    CFL = 1/(N+1)                     # CFL number
    
    if (integration_type == 1):
        Q = N
    elif (integration_type == 2):
        Q = N+1

    wall = 0

    for e, nel in enumerate(Nv):
        
        if (method_type == 'cg'):
            Np = nel*N + 1
            
        elif (method_type == 'dg'):
            Np = nel*(N+1)
    
        # Call of 1D wave solver
        '''
        outputs:
        --------
        qe         : Exact solution
        q          : Numerical solution
        coord      : All grid points
        intma      : Intma(CG/DG)
        '''
        tic = perf_counter()
        
        if(text_case == 'unit'):
            
            qe, q,coord, intma, tf = cg_dgSolver(N,Q,nel, Np, ax, bx, integration_type, g, exactSol,c1, c,u,\
                                CFL, Tfinal, kstages, method_type, diss, icase, alpha, beta, \
                                             ti_method, time_method)
            
            
            # Compute L2- norm
            num = 0
            denom = 0
            error = zeros(Np)
            for i in range(Np):
                num = num + (q[i]-qe[i])**2
                error[i] = abs(q[i]-qe[i])
                denom = denom + (qe[i])**2

            e2 = sqrt(num/denom)

            max_norm[iN, e] = max(error)

            l2e_norm[iN,e] = e2
            Np_array[iN,e] = Np
        
        elif(text_case == 'ice-ocean'):
            
            # For temperature
            cst1 = Tw
            c1 = kT
            bcst1 = Lf/(cw*kT)
            # For salinity
            cst2 = Sw
            c2 = kS
            bcst2 = 1/kS
            
            S, T, coord, intma, tf = ice_ocean_Solver(N,Q,nel, Np, ax, bx, integration_type, hT, hB, \
                                            initial_Temp, initial_Salt, cst1, c1, bcst1,cst2, c2,bcst2, 
                                            Tw, gammaS,gammaT, cw, Lf, cI, TS, b, c, pb, a, Sw, K,M, CFL, Tfinal,\
                                            u, u1,coefF,Meltrate, SaltB, kstages,method_type,ti_method, time_method)
        
        print("\twalltime = {:e}".format(tf))
        
        toc = perf_counter()
        wall += toc - tic
        
        
        #Compute a gridpoint solution
        
        x_sol = zeros(Np)
        for ie in range(1,nel+1):
            for i in range(N+1):
                ip = int(intma[i,ie-1])
                x_sol[ip] = coord[i,ie-1]
        
    timef = Tfinal
    '''
    #qe = exactSol(x_sol, timef, icase)
    if(iplot == True):
        figure(iN)
        plot(x_sol, q, '-', label = 'Computed')
        plot(x_sol, qe, '--', label = 'Exact')
        grid()
        legend()
    '''

In [ ]:
if(text_case == 'ice-ocean'):
    
    figure(2)

    plot(x_sol, T, '-', label = 'T')
    xlabel('x')
    ylabel('Temperature (˚C)')
    title('Temperature profile')
    legend()
    grid()


    figure(3)

    plot(x_sol, S, '--', label = 'S')
    xlabel('x')
    ylabel('Salinity (psu)')
    title('Salinity profile')
    legend()
    grid()
    show()

In [ ]:

if(text_case == 'unit'):
    import cg_graphics
    figure(4)
    clf()

    for i,N in enumerate(order):

        if(N > 3):
            p = polyfit(log(Nv[:2]), log(l2e_norm[i][:2]), 1)
        else:

            p = polyfit(log(Nv), log(l2e_norm[i]), 1)

        loglog(Nv, l2e_norm[i], '-o',markersize=5, label = 'N = {:d}: rate = {:.2f}'.format(N,p[0]))

        loglog(Nv, exp(polyval(p,log(Nv))), '--')

    cg_graphics.set_xticks(Nv)
    xlabel('# Elements')
    ylabel('Error (L2-error)')
    title('Error vs number of Elements ({:s}, {:s})'.format('cg'.upper(), time_method))
    grid(axis='both',linestyle='--')
    legend()
    show()   